In [6]:
%load_ext autoreload
%autoreload 2

from source.config import EXTERNAL_DATA_DIR, INTERIM_DATA_DIR
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Table of contents**<a id='toc0_'></a>    
- [6: Sett opp oversikt over ulike ekvipasjer (bil og henger kombinasjoner) som har blitt benyttet.](#toc1_)    
- [7+8 (mål på hva de foretrekker): Hvor ofte har ulike ekvipasjer blitt benyttet, sett opp statistisk fordeling. + Hvor mange km har ulike ekvipasjer tilbakelagt.](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[6: Sett opp oversikt over ulike ekvipasjer (bil og henger kombinasjoner) som har blitt benyttet.](#toc0_)

In [7]:
df_ekvipasje = pd.read_csv(EXTERNAL_DATA_DIR / 'bil_tilhenger_matching.csv').drop_duplicates(subset='VIN_lastebil')
df_vehicle_data = pd.read_csv(INTERIM_DATA_DIR / "vehicle weight from 74t.csv")

df_vehicle_data = (
    df_vehicle_data[df_vehicle_data['VIN'].isin(df_ekvipasje['VIN_lastebil'])]
    .assign(
        år=pd.to_datetime(df_vehicle_data['Dato'], format='ISO8601').dt.year,
        ekvipasje=df_vehicle_data['VIN'].map(df_ekvipasje.set_index('VIN_lastebil')['ekvipasje'])
    )
)

df6 = (
    df_vehicle_data.groupby(['år', 'ekvipasje']).agg(**{
        'registrerte kjøretøy': ('VIN', 'nunique'),
    })
    .reset_index()
)

df6[['år', 'ekvipasje', 'registrerte kjøretøy']].sort_values(by=['år', 'ekvipasje'], ascending=True)


,år,ekvipasje,registrerte kjøretøy
0,2021,3-akslet trekkvogn med 4-akslet tilhenger,3
1,2021,3-akslet trekkvogn med 5-akslet tilhenger,4
2,2021,4-akslet trekkvogn med 4-akslet tilhenger,1
3,2021,4-akslet trekkvogn med 5-akslet tilhenger,1
4,2022,3-akslet trekkvogn med 4-akslet tilhenger,4
5,2022,3-akslet trekkvogn med 5-akslet tilhenger,7
6,2022,4-akslet trekkvogn med 4-akslet tilhenger,2
7,2022,4-akslet trekkvogn med 5-akslet tilhenger,2
8,2023,3-akslet trekkvogn med 4-akslet tilhenger,4
9,2023,3-akslet trekkvogn med 5-akslet tilhenger,8


In [ ]:
df_antall_unike = (
    df_vehicle_data.groupby(['ekvipasje']).agg(**{
        'registrerte kjøretøy': ('VIN', 'nunique'),
    })
    .reset_index()
)

df_antall_unike[['ekvipasje', 'registrerte kjøretøy']].sort_values(by=['ekvipasje'], ascending=True)

,ekvipasje,registrerte kjøretøy
0,3-akslet trekkvogn med 4-akslet tilhenger,4
1,3-akslet trekkvogn med 5-akslet tilhenger,8
2,4-akslet trekkvogn med 4-akslet tilhenger,2
3,4-akslet trekkvogn med 5-akslet tilhenger,3


In [ ]:
import pandas as pd
import requests
import time

# Funksjon for å hente antall aksler fra NHTSA API
def get_axles(vin):
    url = f"https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVIN/{vin}?format=json"
    try:
        response = requests.get(url)
        data = response.json()
        for item in data["Results"]:
            if item["Variable"] == "Axles":
                return item["Value"]
    except Exception as e:
        print(f"Feil ved VIN {vin}: {e}")
    return "Ukjent"

# Les CSV-fil
df = pd.read_csv("biler.csv")  # Endre til din fil

# Sjekk at "VIN" finnes som kolonne
if "VIN" not in df.columns:
    raise ValueError("CSV-filen må ha en 'VIN'-kolonne!")

# Legg til akselantall basert på VIN
df["Antall aksler"] = df["VIN"].astype(str).apply(get_axles)

# Lagre resultatet til en ny CSV-fil
df.to_csv("biler_med_aksler.csv", index=False)

print("Prosessen er ferdig! Resultatet er lagret i 'biler_med_aksler.csv'.")


# <a id='toc2_'></a>[7+8 (mål på hva de foretrekker): Hvor ofte har ulike ekvipasjer blitt benyttet, sett opp statistisk fordeling. + Hvor mange km har ulike ekvipasjer tilbakelagt.](#toc0_)

Tabellen under viser antall turer og km tilbakelagt for hver ekvipasje for hvert år fra 2021-2024.

In [9]:
df_grouped = df_vehicle_data.groupby(['ekvipasje', 'år']).agg(**{
    'turer': ('VIN', 'count'),
    'kilometer': ('Distanse (km)', 'sum'),
    'registrerte kjøretøy': ('VIN', 'nunique')
}).reset_index()

df_grouped['turer per kjøretøy'] = df_grouped['turer'] / df_grouped['registrerte kjøretøy']
df_grouped['km per kjøretøy'] = df_grouped['kilometer'] / df_grouped['registrerte kjøretøy']

df_grouped.sort_values(by=['år', 'ekvipasje'], ascending=True).round(2)


,ekvipasje,år,turer,kilometer,registrerte kjøretøy,turer per kjøretøy,km per kjøretøy
0,3-akslet trekkvogn med 4-akslet tilhenger,2021,766,282918.0,3,255.33,94306.00
4,3-akslet trekkvogn med 5-akslet tilhenger,2021,941,326593.0,4,235.25,81648.25
8,4-akslet trekkvogn med 4-akslet tilhenger,2021,51,12222.0,1,51.00,12222.00
12,4-akslet trekkvogn med 5-akslet tilhenger,2021,333,137302.0,1,333.00,137302.00
1,3-akslet trekkvogn med 4-akslet tilhenger,2022,1131,374296.0,4,282.75,93574.00
5,3-akslet trekkvogn med 5-akslet tilhenger,2022,1882,568852.0,7,268.86,81264.57
9,4-akslet trekkvogn med 4-akslet tilhenger,2022,504,122606.0,2,252.00,61303.00
13,4-akslet trekkvogn med 5-akslet tilhenger,2022,418,140216.0,2,209.00,70108.00
2,3-akslet trekkvogn med 4-akslet tilhenger,2023,1207,356309.0,4,301.75,89077.25
6,3-akslet trekkvogn med 5-akslet tilhenger,2023,2359,724260.0,8,294.88,90532.50


Tabellen under viser en videre bearbeiding av tabellen ovenfor, gruppert på ekvipasje, aggregert på turer/km per kjøretøy, og delt på antall år.
Dersom man bruker "km per kjøretøy per år" og/eller "turer per kjøretøy per år" som et mål på hvilken ekvipasje som er foretrukket, kommer 4+5 (74T) best ut her.

In [10]:
year_counts = df_grouped.groupby('ekvipasje')['år'].nunique().reset_index()
year_counts = year_counts.rename(columns={'år': 'num_years'})

df_grouped = df_grouped.merge(year_counts, on='ekvipasje', how='left')

df_grouped['turer per kjøretøy per år'] = df_grouped['turer per kjøretøy'] / df_grouped['num_years']
df_grouped['km per kjøretøy per år'] = df_grouped['km per kjøretøy'] / df_grouped['num_years']

df_pivot = df_grouped.pivot_table(
    index='ekvipasje', 
    values=['turer per kjøretøy per år', 'km per kjøretøy per år'], 
    aggfunc='sum'
).reset_index()

df_pivot.sort_values(by='ekvipasje', ascending=True).round(2)

,ekvipasje,km per kjøretøy per år,turer per kjøretøy per år
0,3-akslet trekkvogn med 4-akslet tilhenger,89474.81,284.27
1,3-akslet trekkvogn med 5-akslet tilhenger,80120.02,263.43
2,4-akslet trekkvogn med 4-akslet tilhenger,59539.50,236.00
3,4-akslet trekkvogn med 5-akslet tilhenger,103083.00,301.75
